In [1]:
import numpy as np
import matplotlib.pyplot as plt
import visualization
import os
from gta_math import points_to_homo, ndc_to_view, construct_proj_matrix, view_to_world, construct_view_matrix, ndcs_to_pixels, grid_to_ndc_pcl_linear_view
from visualization import load_depth, save_pointcloud_csv
import progressbar
from pointcloud_to_voxelmap import pointclouds_to_voxelmap
from joblib import Parallel, delayed
from configparser import ConfigParser
from PIL import Image
import pickle
import json
from voxelmaps import camera_to_pointcloud, load_scene_db_data, get_main_image_name, scene_to_voxelmap, get_main_image, scene_to_voxelmap_with_map, ndc_pcl_to_grid_linear_view
import voxelmaps

In [2]:
ini_file = "gta-postprocessing.ini"
visualization.multi_page = False
visualization.ini_file = ini_file

conn = visualization.get_connection_pooled()
cur = conn.cursor()

CONFIG = ConfigParser()
CONFIG.read(ini_file)
in_directory = CONFIG["Images"]["Tiff"]
out_directory = CONFIG["Images"]["MlDatasetVoxel"]
out_inspect_directory = r'D:\showing-pointclouds'

#voxelmaps.MAX_DISTANCE = 500
voxelmaps.MAX_DISTANCE = 25

In [4]:
#scene_id = '623b6ace-2a9c-4067-8c5f-42cb46d56f53' # from 4cameras on car run, I think
scene_id = '386b407b-586c-4d88-9d41-8dc2a0b70e70' # from voxelmap run

cameras = load_scene_db_data(scene_id)

base_name = get_main_image_name(cameras)
voxels, values, map_voxel_size, map_obj = scene_to_voxelmap_with_map(cameras)
occupied_voxels = voxels[:, values >= 0]
pointcloud = camera_to_pointcloud(get_main_image(cameras))

In [5]:
cam = get_main_image(cameras)
proj_matrix = cam['proj_matrix']
view_matrix = cam['view_matrix']
width = cam['width']
height = cam['height']

In [6]:
x_min = -1
x_max = 1
y_min = -1
y_max = 1
x_range = 240
y_range = 160
z_meters_min = 1.5
z_meters_max = voxelmaps.MAX_DISTANCE
# z min calc
z_min = proj_matrix @ [1, 1, -z_meters_max, 1]
z_min = z_min[2] / z_min[3]
# z max calc
z_max = proj_matrix @ [1, 1, -z_meters_min, 1]
z_max = z_max[2] / z_max[3]
z_range = 100
#z_range = 2000
x_bin = (x_max - x_min) / x_range
y_bin = (y_max - y_min) / y_range
z_bin = (z_max - z_min) / z_range


In [7]:
# now I need to calculate correct Z points, deformed in NDC, linear in view frustum
z_view_bin = (z_meters_max - z_meters_min) / z_range
#X_view, Y_view, Z_view, W_view = np.mgrid[1:2:1, 1:2:1, -z_meters_max:-z_meters_min+1e-7:z_view_bin, 1:2:1]  # W is here as homo coordinate
X_view, Y_view, Z_view, W_view = np.meshgrid(np.linspace(1,2,1), np.linspace(1,2,1), np.linspace(-z_meters_max,-z_meters_min,z_range), np.linspace(1,2,1))
view_positions = np.vstack([X_view.ravel(), Y_view.ravel(), Z_view.ravel(), W_view.ravel()])
ndc_positions = proj_matrix @ view_positions
ndc_positions /= ndc_positions[3, :]
ndc_z = ndc_positions[2, :]
ndc_z_tiled = np.tile(ndc_z,(x_range, y_range, 1))[:, :, :, np.newaxis]

In [8]:
#X, Y, Z, W = np.mgrid[x_min:x_max:x_bin, y_min:y_max:y_bin, z_min:z_max:z_bin, 1:2:1]  # W is here as homo coordinate
X, Y, Z, W = np.meshgrid(np.linspace(x_min,x_max,x_range), np.linspace(y_min,y_max,y_range), np.linspace(z_min,z_max,z_range), np.linspace(1,2,1))
positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()])
positions_fixed = np.vstack([X.ravel(), Y.ravel(), ndc_z_tiled.ravel(), W.ravel()])

In [11]:
params = {
    'width': width,
    'height': height,
    'proj_matrix': proj_matrix,
}

points_view = ndc_to_view(positions, proj_matrix)
points_world = view_to_world(points_view, view_matrix)
# here I find corresponding voxels for generated points, by obtaining voxelmap reference
voxel_values = map_obj.get_voxels(points_world[0:3, :], np.zeros((points_world.shape[1], 1)))
occupied_selected_voxels_view = points_view[:, voxel_values > 0]
occupied_ndc_positions = positions[:, voxel_values > 0]

w = np.ones((1,occupied_voxels.shape[1]))
voxels_homo = np.vstack([occupied_voxels, w])
voxels_view = view_matrix @ voxels_homo
w = np.ones((1,pointcloud.shape[1]))
pointcloud_homo = np.vstack([pointcloud, w])
pointcloud_view = view_matrix @ pointcloud_homo

points_view_fixed = ndc_to_view(positions_fixed, proj_matrix)
points_world_fixed = view_to_world(points_view_fixed, view_matrix)
voxel_values_fixed = map_obj.get_voxels(points_world_fixed[0:3, :], np.zeros((points_world_fixed.shape[1], 1)))
occupied_selected_voxels_view_fixed = points_view_fixed[:, voxel_values_fixed > 0]
occupied_ndc_positions_fixed = positions_fixed[:, voxel_values_fixed > 0]

occupied_ndc_grid = ndc_pcl_to_grid_linear_view(x_range, y_range, z_range, occupied_ndc_positions_fixed, proj_matrix, z_meters_min, z_meters_max)
occupied_ndc_positions_reconstructed = grid_to_ndc_pcl_linear_view(occupied_ndc_grid, proj_matrix, z_meters_min, z_meters_max)
occupied_ndc_positions_reconstructed = np.hstack((occupied_ndc_positions_reconstructed, np.ones((occupied_ndc_positions_reconstructed.shape[0], 1)))).T

points_view_reconstructed = ndc_to_view(occupied_ndc_positions_reconstructed, proj_matrix)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in greater
  # This is added back by InteractiveShellApp.init_path()
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in greater
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in greater
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater


In [12]:
save_pointcloud_csv(points_view.T[:, 0:3], '{}/{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(points_view_fixed.T[:, 0:3], '{}/fixed-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(positions.T[:, 0:3], '{}/ndc-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(occupied_selected_voxels_view.T[:, 0:3], '{}/voxels-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(voxels_view.T[:, 0:3], '{}/orig-voxels-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(occupied_ndc_positions.T[:, 0:3], '{}/ndc-occupied-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(pointcloud_view.T[:, 0:3], '{}/orig-pointcloud-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(occupied_selected_voxels_view_fixed.T[:, 0:3], '{}/voxels-fixed-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(occupied_voxels.T[:, 0:3], '{}/all-voxels-{}.csv'.format(out_inspect_directory, base_name))
save_pointcloud_csv(points_view_reconstructed.T[:, 0:3], '{}/reconstructed-{}.csv'.format(out_inspect_directory, base_name))


In [ ]:
occupied_ndc_grid.tofile('{}/{}.bin'.format(out_inspect_directory, base_name), sep=';')
np.save('{}/{}'.format(out_inspect_directory, base_name), occupied_ndc_grid)


In [ ]:
# now I create x X y X z grid with 0s and 1s as grid
# so now I have data in pointcloud. And I need to convert these NDC values
# into indices, so x:[-1, 1] into [0, 239], y:[-1, 1] to [0, 159], 
# and z:[z_min, z_max] into [0, 99]
voxelmap_ndc_grid = np.zeros((x_range, y_range, z_range), dtype=np.bool)
vecs = ndcs_to_pixels(occupied_ndc_positions[0:2, :], (y_range, x_range))
vec_y = vecs[0, :]
vec_x = vecs[1, :]
vec_z = ((occupied_ndc_positions[2, :] - z_min) / z_bin).astype(np.int32)
voxelmap_ndc_grid[vec_x, vec_y, vec_z] = 1


In [ ]:
voxels.shape

In [ ]:
w = np.ones((1,voxels.shape[1]))

In [ ]:
voxels_homo.shape

In [ ]:
points_world[0:3, :].shape

In [ ]:
positions.T[:, 0:3].shape

In [ ]:
occupied_voxels.T[:, 0:3].shape

In [ ]:
#rgb_img_name = os.path.join(r'D:\generic-dataset', '{}.jpg'.format('2018-03-30--06-00-56--114'))
rgb_img_name = os.path.join(in_directory, '{}.tiff'.format(base_name))
im = Image.open(rgb_img_name)
im = im.convert(mode="RGB")
arr = np.array(im)

plt.figure(figsize=(20, 20))
plt.axis('off')
plt.imshow(arr)
plt.show()

In [ ]:
depth = load_depth(base_name)

plt.figure(figsize=(20, 20))
plt.axis('off')
plt.imshow(depth, cmap='gray')
plt.show()

In [ ]:
for camera in cameras:
    depth = load_depth(camera['imagepath'])
    plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(depth, cmap='gray')
    plt.show()

In [ ]:
for camera in cameras:
    rgb_img_name = os.path.join(in_directory, '{}.tiff'.format(camera['imagepath']))
    im = Image.open(rgb_img_name)
    im = im.convert(mode="RGB")
    arr = np.array(im)
    plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(arr)
    plt.show()

In [ ]:
base_name

In [ ]:
display(occupied_ndc_positions.min(axis=1))
display(occupied_ndc_positions.max(axis=1))
occupied_ndc_positions.shape


In [ ]:
# for debug purposes, now I visualize the voxelmap in NDC as image
new_depth = np.zeros((y_range, x_range))
vecs = ndcs_to_pixels(occupied_ndc_positions_fixed[0:2, :], (y_range, x_range))
vec_y = vecs[0, :]
vec_x = vecs[1, :]
vec_z = occupied_ndc_positions_fixed[2, :]
new_depth[vec_y, vec_x] = vec_z

%matplotlib notebook

plt.figure(figsize=(10, 6))
plt.axis('off')
plt.imshow(new_depth, cmap='gray')
plt.show()

### now image from bool grid, which will be directly output of neural network

In [ ]:
occupied_ndc_grid.shape
np.argmax(occupied_ndc_grid == True, axis=2)[100, 100]
occupied_ndc_grid[150, 80, :]


In [ ]:
occupied_ndc_grid = np.flip(occupied_ndc_grid, axis=2)

# now I have just boolean for each value
# so I create mask to assign higher value to booleans in higher index
target_width = occupied_ndc_grid.shape[0]
target_height = occupied_ndc_grid.shape[1]
depth_size = occupied_ndc_grid.shape[2]

new_depth = np.argmax(occupied_ndc_grid, axis=2)
new_depth = new_depth.T

new_depth *= int(255/depth_size)

%matplotlib notebook

plt.figure(figsize=(10, 7))
plt.axis('off')
plt.imshow(new_depth, cmap='gray')
plt.show()

In [ ]:
print(new_depth.max())
print(new_depth.min())


In [ ]:
display(vecs.min(axis=1))
display(vecs.max(axis=1))
vecs.shape


In [ ]:
vec_x.max()

In [ ]:
occupied_ndc_positions.max(axis=1)
np.unique(occupied_ndc_positions[2, :]).shape

In [ ]:
np.unique(occupied_ndc_positions[2, :])

In [ ]:
np.unique(occupied_ndc_positions[2, :]) * z_range

In [ ]:
(np.unique(occupied_ndc_positions[2, :]) - z_min) * ((z_range-1) / (z_max - z_min))

In [ ]:
print(z_bin)
print(z_min)
print(z_max)
print(z_min / z_bin)
print(z_max / z_bin)
print(z_max - z_min)

In [ ]:
(np.unique(occupied_ndc_positions[2, :]) - z_min) / z_bin

In [ ]:
np.size(voxelmap_ndc_grid) #181289, 3840000

In [ ]:
x_range * y_range * z_range

In [ ]:
voxelmap_ndc_grid.shape

### playing with depths projections

In [ ]:
z_meters_min = 1.5
z_meters_max = voxelmaps.MAX_DISTANCE
# z min calc
z_min = proj_matrix @ [1, 1, -z_meters_max, 1]
z_min = z_min[2] / z_min[3]
# z max calc
z_max = proj_matrix @ [1, 1, -z_meters_min, 1]
z_max = z_max[2] / z_max[3]
z_range = 100
z_bin = (z_max - z_min) / (z_range)
#X, Y, Z, W = np.mgrid[1:2:1, 1:2:1, z_min:z_max:z_bin, 1:2:1]  # W is here as homo coordinate
X, Y, Z, W = np.meshgrid(np.linspace(1,2,1), np.linspace(1,2,1), np.linspace(z_min,z_max,z_range), np.linspace(1,2,1))
positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()])
points_view = ndc_to_view(positions, proj_matrix)

In [ ]:
positions[2, :].shape

In [ ]:
plt.figure(figsize=(15,15))
plt.plot(positions[2, :], points_view[2, :])
plt.show()

In [ ]:
print(positions[2, :].max())
print(positions[2, :].min())
print(points_view[2, :].max())
print(points_view[2, :].min())


In [ ]:
z_bin = (z_meters_max - z_meters_min) / (z_range)
#X, Y, Z, W = np.mgrid[1:2:1, 1:2:1, -z_meters_max:-z_meters_min:z_bin, 1:2:1]  # W is here as homo coordinate
X, Y, Z, W = np.meshgrid(np.linspace(1,2,1), np.linspace(1,2,1), np.linspace(-z_meters_max,-z_meters_min,z_range), np.linspace(1,2,1))
view_positions = np.vstack([X.ravel(), Y.ravel(), Z.ravel(), W.ravel()])
ndc_positions = proj_matrix @ view_positions
ndc_positions /= ndc_positions[3, :]


In [ ]:
plt.figure(figsize=(15,15))
plt.plot(view_positions[2, :], ndc_positions[2, :])
plt.show()

In [ ]:
print(ndc_positions[2, :].max())
print(ndc_positions[2, :].min())
print(view_positions[2, :].max())
print(view_positions[2, :].min())


In [ ]:
view_positions[2, :].shape

In [ ]:
list(np.mgrid[-z_meters_max:-z_meters_min+1e-7:z_bin])

In [ ]:
view_positions[2, :]

In [ ]:
z_bin

In [ ]:
-1.9850000000000563 + z_bin

In [ ]:
-z_meters_max + (z_bin * 99)

In [ ]:
ndc_positions[2, :]

# new NDC sampling, with nonlinear Z axis which will be mapped to linear distances in view frustum

In [ ]:
z_meters_min = 1.5
z_meters_max = voxelmaps.MAX_DISTANCE
# z min calc
z_min = proj_matrix @ [1, 1, -z_meters_max, 1]
z_min = z_min[2] / z_min[3]
# z max calc
z_max = proj_matrix @ [1, 1, -z_meters_min, 1]
z_max = z_max[2] / z_max[3]
z_range = 100
z_bin = (z_max - z_min) / z_range
#X, Y, Z, W = np.mgrid[1:2:1, 1:2:1, z_min:z_max+1e-7:z_bin, 1:2:1]  # W is here as homo coordinate
X, Y, Z, W = np.meshgrid(np.linspace(1,2,1), np.linspace(1,2,1), np.linspace(z_min,z_max,z_range), np.linspace(1,2,1))
positions = np.vstack([X.ravel(), Y.ravel(), ndc_positions[2, :], W.ravel()])
points_view = ndc_to_view(positions, proj_matrix)

In [ ]:
plt.figure(figsize=(15,15))
plt.plot(np.linspace(0, 1, len(points_view[2, :])), points_view[2, :])
plt.show()

In [ ]:
ndc_z_tiled = np.tile(ndc_z,(240, 160, 1))[:, :, :, np.newaxis]

In [ ]:
Z.shape

In [ ]:
Z[:].shape

In [ ]:
Z[0, :, 0, 0]

In [ ]:
ndc_z_tiled[0, :, 0, 0]

In [ ]:
Z - ndc_z_tiled

In [ ]:
zmax = 50
zmin = 1.5
z_bin = (zmax - zmin) / (z_range - 1)
print(np.mgrid[-zmax:-zmin:z_bin])
print(np.mgrid[-zmax:-zmin:z_bin].shape)


In [ ]:
zmax = 30
zmin = 1.5
z_bin = (zmax - zmin) / (z_range - 1)
print(np.mgrid[-zmax:-zmin:z_bin])
print(np.mgrid[-zmax:-zmin:z_bin].shape)


In [ ]:
zmax = 30
zmin = 1.5
print(np.linspace(-zmax, -zmin, z_range))
print(np.linspace(-zmax, -zmin, z_range).shape)

In [ ]:
zmax = 50
zmin = 1.5
print(np.linspace(-zmax, -zmin, z_range))
print(np.linspace(-zmax, -zmin, z_range).shape)

In [ ]:
%matplotlib inline
rgb_im = Image.open(os.path.join(out_directory, '2018-03-07--17-46-32--369.jpg'))
plt.figure(figsize=(10, 7))
plt.axis('off')
plt.imshow(rgb_im)
plt.show()

rgb_im = Image.open(os.path.join(out_directory, '2018-03-07--18-01-55--862.jpg'))
plt.figure(figsize=(10, 7))
plt.axis('off')
plt.imshow(rgb_im)
plt.show()
